# Analyzing individual questions from the dataset

In [39]:
# Library imports
import os

import pandas as pd
import numpy as np
from datasets import load_dataset
import matplotlib.pyplot as plt
from sentence_transformers import util

In [40]:
# Global variables
MODEL = 'multi-qa-mpnet-base-dot-v1'
DATA_PATH = '../data'
DATA_EMBEDDINGS_PATH = f'../data_embeddings/{MODEL}'
DATASET_EMBEDDINGS_PATH = f'../dataset_embeddings/{MODEL}'
DATASET_NAME = 'Rivert97/ug-normativity'

# Other Variables
filename = 'ley-organica-de-la-universidad-de-guanajuato'
#filename = 'estatuto-organico-de-la-universidad-de-guanajuato'
#filename = 'reglamento-de-mecanismos-alternos-de-solucion-de-controversias-ug-ugto'

In [41]:
# Loading the data
data = pd.read_csv(os.path.join(DATA_PATH, f"{filename}.csv"), sep=',')
embeddings = pd.read_csv(os.path.join(DATA_EMBEDDINGS_PATH, f"{filename}.csv"), sep=',', index_col=0)

In [42]:
# Loading questions dataset
dataset = load_dataset(DATASET_NAME)
dataset = dataset['train']
dataset = dataset.filter(lambda row: row['title'] == filename)

questions_embeddings = pd.read_csv(os.path.join(DATASET_EMBEDDINGS_PATH, f"{filename}.csv"), sep=',', index_col=0)

# Analyzing Question

In [43]:
# Select question
question = dataset[1]
q_embedding = questions_embeddings.loc[question['id']]

print("ID:", question['id'])
print("Sentence:", question['question'])
print("Embedding:", q_embedding)

ID: 28738544e42b076370fc813c5711852a
Sentence: ¿Cuál es el fundamento legal que sustenta la Ley Orgánica?
Embedding: 0     -0.353499
1      0.290611
2     -0.296301
3      0.106464
4      0.137439
         ...   
763    0.156694
764   -0.138280
765   -0.353606
766   -0.132102
767    0.200108
Name: 28738544e42b076370fc813c5711852a, Length: 768, dtype: float64


In [46]:
# Get nearest neighbors
k = 5
scores = util.dot_score(q_embedding.values, embeddings.values)
doc_score_pairs = list(zip(data.index.tolist(), scores[0]))

doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

context_in_chunks = False
for pair in doc_score_pairs[:k]:
    chunk = data.loc[pair[0]]

    if chunk['path'].lower().endswith(question['context'].lower()):
        context_in_chunks = True
        print("ANSWER:")

    print("Distance:", pair[1])
    print("Chunk text:", chunk['sentences'])
    print("Parent:", chunk['parent'])
    print()


print(f"Answer in Top-{k}?", context_in_chunks)


Distance: tensor(28.7596, dtype=torch.float64)
Chunk text: Se abroga la Ley Orgánica de la Universidad de Guanajuato, contenida en el Decreto número 279, emitido por la Quincuagésima Quinta Legislatura del Estado de Guanajuato, publicado en el Periódico Oficial del Gobierno del Estado número 39, de fecha 17 de mayo de 1994.
Parent: ARTÍCULO SEGUNDO

ANSWER:
Distance: tensor(28.6993, dtype=torch.float64)
Chunk text: Esta Ley, el Estatuto Orgánico, sus reglamentos y las disposiciones de carácter general que de ella deriven, se sujetarán a lo dispuesto en el artículo 3º de la Constitución Política de los Estados Unidos Mexicanos y demás ordenamientos aplicables.
Parent: Artículo 2

Distance: tensor(25.4856, dtype=torch.float64)
Chunk text: El Consejo General Universitario, con base a lo establecido en la Ley de la materia, incluirá en el Estatuto Orgánico un apartado sobre la responsabilidad del personal administrativo de la Universidad, el cual contendrá sus obligaciones específicas. El 